<a href="https://colab.research.google.com/github/jordanlauritzen2/CRFR_Sp24_Project/blob/main/CRFR_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assessing Fire and Rescue Service Strain to Anticipate Emerging Service Demands

## Files Required:

The following files (all located in the 'Data' folder) are required for the notebook to run:

*   2012_2021_NewData.xlsx
*   Acutal_ValueBY_Year.csv
*   HomeType.csv
*   FIREDISTRICT.shp **
*   FIRESTATIONS.shp **
*   FIRESTATIONZONES.shp **
*   Parcel.shp **
*   Blockgroups.shp **
*   Population files *
      *   Contains 9 csv files for 2013-2021
*   Homes65+ files *
      *   Contains 9 csv files for 2013-2021
*   Blockgroups files *
      *   Contains 10 .shp files


<br>
<br>
<br>

*Entire folder is required



** For all shapefiles `.shp` the corresponding `.cpg`, `.dbf`, `.prj`, `.shx`, `.xml` are required

## Roles of Team Members

Glen:
> Words

Sophie
> Words

Jordan:
> Words

# Abstract

Words

# Introduction

Words

# Methods

Words

# Results and Discussion

this is copied from previous project

In [1]:
!pip install geopandas -q
!pip install mapclassify -q
!pip install contextily -q
!pip install cpi -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 14.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import mapclassify
import contextily as cx
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import seaborn as sns
from sklearn.linear_model import LinearRegression
from cpi import inflate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import statsmodels.api as sm
from calendar import month_abbr
from functools import reduce
import scipy as sp
from statsmodels.formula.api import ols

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cwd = os.getcwd()
pathCRFR = cwd + '/drive/My Drive/Colab Notebooks/Math Clinic'
pathFournier = 'Math Clinic/2024sp/CRFR/Data/'
if os.path.exists(pathCRFR + pathFournier):
    pathCRFR += pathFournier
os.listdir(pathCRFR) # lists files in the drive to check

['Acutal_ValueBY_Year.csv',
 'HomeType.csv',
 'Incident_Data_CLEAN.xlsx',
 '2012_2021_NewData.xlsx',
 'Blockgroups',
 'Population',
 'ShapeFiles',
 'Homes65+']

In [5]:
filepath = pathCRFR

## Importing Data

### Incident Data

In [ ]:
incident_data = pd.read_excel(filepath + '2012_2021_NewData.xlsx').drop(columns = 'Jurisdiction Name')

In [ ]:
# Columns with spaces rewritten with _ for calling them
incident_data = incident_data.rename(columns = {'Incident Year':'incident_year',
                                'Incident Number':'incident_number',
                                'Date of Call':'date_of_call',
                                'Day of Week':'day_of_week',
                                'Time of Call':'time_of_call',
                                'Hour of Day':'hour_of_day',
                                'Incident Type':'incident_type',
                                'Incident Type Category':'incident_type_category',
                                'Location from CAD':'location_from_cad',
                                'Response Zone':'response_zone',
                                'Station District':'station_district'})

### Actual Value Data (home values)

In [ ]:
actual_value_df = pd.read_csv(filepath + 'Acutal_ValueBY_Year.csv', sep = ',')

In [ ]:
actual_value_df = actual_value_df.replace('<Null>', np.nan) # Changes <Null> to NaN that pandas recognizes
actual_value_df.drop(actual_value_df.columns[[13,14]],axis=1,inplace=True) # Drops empty columns 13, 14
actual_value_df = actual_value_df.dropna() # Drops empty rows

### Home Type

SFD = Single Family Dwelling \\
SFA = Service Family Accomodation \\
MF = Multifamily dwelling \\

In [ ]:
home_type_df = pd.read_csv(filepath + 'HomeType.csv', sep = ',').dropna()

### Population Tables (census data from past project)

### Age Demographics (census data from past project)

### Shape Files

# Conclusion

Words